# Start

In [4]:
!aws codeartifact login --tool pip --domain cirrus-ml-ds-domain --domain-owner 813736554012 --repository cirrus-ml-ds-shared-repo
!pip install awswrangler --quiet
!pip install lightgbm --quiet
# !pip install category_encoders --quiet
# !pip install imbalanced-learn --quiet

Successfully configured pip to use AWS CodeArtifact repository https://cirrus-ml-ds-domain-813736554012.d.codeartifact.eu-north-1.amazonaws.com/pypi/cirrus-ml-ds-shared-repo/ 
Login expires in 12 hours at 2022-10-13 20:18:29+00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda 4.14.0 requires ruamel_yaml_conda>=0.11.14, which is not installed.
docker-compose 1.29.2 requires PyYAML<6,>=3.10, but you have pyyaml 6.0 which is incompatible.


In [5]:
import boto3
import awswrangler as wr
import pandas as pd
from lightgbm import LGBMClassifier
import numpy as np


boto3.setup_default_session(region_name="eu-north-1")

# Check Train & Test data

## 1- preprocessing results

In [11]:
# test_preprocessed_data_path = "s3://mlops-dev-370702650160-eu-north-1-data/lifecycle/60d/customerone-inf/b4caacc/2022_10_05_15_30_26/p1033/output/training/test/test.csv"
# train_preprocessed_data_path = "s3://mlops-dev-370702650160-eu-north-1-data/lifecycle/60d/customerone-inf/b4caacc/2022_10_05_15_30_26/p1033/output/training/train/train.csv"

test_preprocessed_data_path = "s3://mlops-dev-370702650160-eu-north-1-data/lifecycle/60d/customerone-inf/200c88a/2022_10_12_16_24_11/p1033/output/training/test/test.csv"
train_preprocessed_data_path = "s3://mlops-dev-370702650160-eu-north-1-data/lifecycle/60d/customerone-inf/200c88a/2022_10_12_16_24_11/p1033/output/training/train/train.csv"

test_preprocessed_df = pd.read_csv(test_preprocessed_data_path)
train_preprocessed_df = pd.read_csv(train_preprocessed_data_path)

### Test data

In [9]:
test_preprocessed_df
# test_preprocessed_df['tgt_xsell_cust_voice_to_fixed']
# test_preprocessed_df['tgt_xsell_cust_voice_to_fixed'].unique()

,customer_id,current_dt,tgt_xsell_cust_voice_to_fixed,dmgrphc_b_age_bucket_txt,dmgrphc_b_zip_code_cd,dmgrphc_b_gender_typ,dmgrphc_b_city_txt,dmgrphc_b_country_txt,dmgrphc_b_no_of_dependents_val,dmgrphc_b_habits_desc_txt,...,cust_prd_hld_d_tvchannelpackage_active_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_vas_closed_0_to_30_over_30_to_60_days_avg_val,rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg,rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg,rev_m_eom_total_bill_amt_m0_val,rev_m_eom_total_bill_amt_m1_val,rev_m_eom_total_bill_amt_m2_val,partition_0,iteration_id,split
0,100608,2021-08-26,0,58,73,M,Katherinechester,Sri Lanka,3,Bad,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TEST
1,100669,2021-09-06,0,39,22,F,Larsentown,Uzbekistan,1,Bad,...,1.000000,NaN,0.0,0.0,0.0,0.0,0.0,xsell_cust_voice_to_fixed,0,TEST
2,100109,2021-08-22,0,46,10,F,Lake Christophertown,Martinique,4,Bad,...,2.214286,NaN,NaN,NaN,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TEST
3,100109,2021-08-22,1,46,10,F,Lake Christophertown,Martinique,4,Bad,...,2.214286,NaN,NaN,NaN,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TEST
4,100109,2021-08-22,0,46,10,F,Lake Christophertown,Martinique,4,Bad,...,2.214286,NaN,NaN,NaN,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TEST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1237,10059,2021-08-17,0,59,63,U,West Ashlee,Canada,3,Good,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TEST
1238,10059,2021-08-17,0,59,63,U,West Ashlee,Canada,3,Good,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TEST
1239,10059,2021-08-17,0,59,63,U,West Ashlee,Canada,3,Good,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TEST
1240,10059,2021-08-17,0,59,63,U,West Ashlee,Canada,3,Good,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TEST


In [12]:
test_features

,rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg,rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg,dmgrphc_b_zip_code_cd,cust_prd_hld_d_mobilevoicesubscription_active_30_to_60_days_avg_val,cust_prd_hld_d_tvchannelpackage_active_30_to_60_days_avg_val,cust_prd_hld_d_mobilevoicesubscription_active_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_vas_active_30_to_60_days_avg_val,cust_prd_hld_d_fixedbroadband_closed_val,cust_prd_hld_d_fixedbroadband_closed_30_to_60_days_avg_val,cust_prd_hld_d_fixedbroadband_active_30_to_60_days_avg_val,...,ci_d_variation_sum_total_resolved_cmpl_last_30_to_60_days_val,cust_prd_hld_d_postpaid_fixedbroadband_cnt,cust_prd_hld_d_mobilevoicesubscription_closed_0_to_30_days_avg_val,cust_prd_hld_d_mobilevoicesubscription_closed_0_to_30_over_30_to_60_days_avg_val,ci_d_compl_last_0_to_30_avg_days_settle_cnt,ci_d_sum_total_num_of_interactions_last_0_to_60_days_cnt,cust_prd_hld_d_vas_closed_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_vas_active_0_to_30_days_avg_val,ci_d_total_num_of_interactions_weekday_last_0_to_30_cnt,cust_prd_hld_d_fixedbroadband_closed_0_to_30_days_avg_val
0,NaN,NaN,73,0.0,0.000000,NaN,2.322581,0,0.0,0.000000,...,NaN,1,0.0,NaN,0.0,0,NaN,2.774194,0,0.0
1,0.0,0.0,22,0.0,1.000000,NaN,1.000000,0,0.0,0.000000,...,NaN,0,0.0,NaN,0.0,0,NaN,1.419355,0,0.0
2,NaN,NaN,10,1.0,0.451613,1.0,1.000000,0,0.0,0.000000,...,NaN,1,0.0,NaN,0.0,0,NaN,1.000000,0,0.0
3,NaN,NaN,10,1.0,0.451613,1.0,1.000000,0,0.0,0.000000,...,NaN,1,0.0,NaN,0.0,0,NaN,1.000000,0,0.0
4,NaN,NaN,10,1.0,0.451613,1.0,1.000000,0,0.0,0.000000,...,NaN,1,0.0,NaN,0.0,0,NaN,1.000000,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1237,NaN,NaN,63,0.0,0.000000,NaN,1.000000,0,0.0,0.064516,...,NaN,4,0.0,NaN,0.0,0,NaN,1.000000,0,0.0
1238,NaN,NaN,63,0.0,0.000000,NaN,1.000000,0,0.0,0.064516,...,NaN,4,0.0,NaN,0.0,0,NaN,1.000000,0,0.0
1239,NaN,NaN,63,0.0,0.000000,NaN,1.000000,0,0.0,0.064516,...,NaN,4,0.0,NaN,0.0,0,NaN,1.000000,0,0.0
1240,NaN,NaN,63,0.0,0.000000,NaN,1.000000,0,0.0,0.064516,...,NaN,4,0.0,NaN,0.0,0,NaN,1.000000,0,0.0


In [14]:
[print(f"col={col}, unique={test_preprocessed_df[col].unique()}") for col in test_preprocessed_df.columns];

col=customer_id, unique=[100608 100669 100109 100215 100488 100874 100979 100149  10022 100776
  10059 100753 100224 100105 100954 100692 100783 100569 100620 100310
 100165 100207 100552 100679 100938 100277 100404 100698 100127 100200
 100397 100339 100791 100143 100701 100158 100885 100370 100435 100623
 100249 100591 100742 100152 100673  10090  10056 100964 100194 100771
 100117 100356 100131 100343 100634  10091 100531 100210 100487 100817
   1002 100551 100779 100899 100984 100599 100801 100256 100522 100705
 100976 100565 100983 100121 100389 100107 100616 100287 100485 100384
 100505  10064 100650 100737   1004 100233  10085 100426 100514  10074
 100736 100217 100612 100526 100914 100170 100666 100350 100369  10083
 100371  10057 100306 100164  10051  10094 100125 100842 100490 100141
 100894 100195 100251 100985  10095 100865 100835 100190 100230 100421
   1008 100921 100331 100662 100595 100943 100580  10071 100523  10061
 100492 100703 100573 100687 100562 100192 100271 100

In [30]:
test_features = pd.read_csv("s3://mlops-dev-370702650160-eu-north-1-data/lifecycle/60d/customerone-inf/200c88a/2022_10_12_16_24_11/p1033/output/training/test/test_features.csv")
# cols = list(test_features.columns)
# test_features[cols]= test_features[cols].fillna(0,)

# [print(f"col={col}, unique={train_preprocessed_df[col].unique()}") for col in test_features.columns];
test_features['tgt_xsell_cust_voice_to_fixed']

KeyError: 'tgt_xsell_cust_voice_to_fixed'

### Trian data

In [19]:
train_preprocessed_df
# train_preprocessed_df.columns
# train_preprocessed_df.values

,customer_id,current_dt,tgt_xsell_cust_voice_to_fixed,dmgrphc_b_age_bucket_txt,dmgrphc_b_zip_code_cd,dmgrphc_b_gender_typ,dmgrphc_b_city_txt,dmgrphc_b_country_txt,dmgrphc_b_no_of_dependents_val,dmgrphc_b_habits_desc_txt,...,cust_prd_hld_d_tvchannelpackage_active_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_vas_closed_0_to_30_over_30_to_60_days_avg_val,rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg,rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg,rev_m_eom_total_bill_amt_m0_val,rev_m_eom_total_bill_amt_m1_val,rev_m_eom_total_bill_amt_m2_val,partition_0,iteration_id,split
0,100165,2021-07-06,0,39,67,U,Port Michelle,Cocos (Keeling) Islands,1,Good,...,1.000000,NaN,0.0,0.0,0.0,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN
1,100979,2021-07-21,0,69,21,U,Morrisshire,Reunion,3,Good,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TRAIN
2,100979,2021-07-21,0,69,21,U,Morrisshire,Reunion,3,Good,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TRAIN
3,100979,2021-07-21,0,69,21,U,Morrisshire,Reunion,3,Good,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TRAIN
4,100979,2021-07-21,1,69,21,U,Morrisshire,Reunion,3,Good,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TRAIN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4183,100488,2021-06-10,0,34,81,F,Hansenbury,Greece,3,Bad,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN
4184,100569,2021-05-26,0,52,37,U,South Christineton,Samoa,2,Bad,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TRAIN
4185,100580,2021-07-18,0,33,88,F,East Tina,Bhutan,0,Good,...,2.384615,NaN,0.0,0.0,0.0,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN
4186,100709,2021-05-27,0,37,39,U,South Sarah,Brunei Darussalam,2,Bad,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TRAIN


In [24]:
[print(f"col={col}, unique={train_preprocessed_df[col].unique()}") for col in train_preprocessed_df.columns];

col=customer_id, unique=[100165 100979 100170 100472 100771 100551 100889 100642 100676 100117
 100492  10083 100415 100791 100844 100109 100138  10023 100459 100372
 100421 100865 100919  10025 100375 100540 100921 100280 100353 100370
  10072 100893 100556 100749 100842 100843 100898 100200 100277 100298
  10037 100452 100744 100947 100435 100490 100520 100895 100190 100256
  10056 100816 100894 100588 100952 100287 100388  10052 100736 100286
 100916 100323 100776 100306 100765 100912 100361 100534 100901  10012
 100393 100505 100585 100704 100239 100616 100860 100124 100392 100559
  10090 100344 100526 100583 100257  10059  10060 100701   1004 100759
 100168 100832 100404 100522 100552 100608 100336 100519 100249 100668
 100569 100709 100343 100826 100950 100127  10055 100141 100488 100523
 100580 100414 100565 100748 100536 100696 100737 100807 100314 100679
 100473 100318 100728 100763 100874 100152 100217 100590 100191 100351
 100730 100149 100326 100669 100433 100706 100225 100

In [33]:
bucket='s3://mlops-dev-370702650160-eu-north-1-data/lifecycle/60d/customerone-dev-branch/no-revision-provided/2022_09_03_09_52_32/p1033/output/training/processed/training'
X = 'train.csv' 
X_path = 's3://{}/{}'.format(bucket, X)

df = pd.read_csv(X_path)
print(df.shape) # (4187, 76)
df.head()

(4187, 76)


,100421,2021-07-28,1,68,33,M,Andrewmouth,New Caledonia,2,Good,...,1.1071428571428572,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,xsell_cust_voice_to_fixed,0.14,TRAIN
0,100620,2021-07-27,0,26,70,F,West Samantha,Qatar,0,Bad,...,15.5,NaN,0.0,0.0,0.0,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN
1,100738,2021-07-09,1,21,78,F,Adamsshire,Madagascar,3,Bad,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TRAIN
2,100771,2021-06-30,0,27,0,U,Velazquezfurt,Azerbaijan,1,Good,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN
3,100421,2021-08-03,1,68,33,M,Andrewmouth,New Caledonia,2,Good,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,xsell_cust_voice_to_fixed,0,TRAIN
4,100488,2021-06-28,0,34,81,F,Hansenbury,Greece,3,Bad,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,xsell_cust_voice_to_fixed,0,TRAIN


In [34]:
X_train_fold_0 = '0/train_x.csv' 
X_train_fold_0_path = 's3://{}/{}'.format(bucket, X_train_fold_0)

df_X_train_fold_0 = pd.read_csv(X_train_fold_0_path, )
print(df_X_train_fold_0.shape)
df_X_train_fold_0.head()

(2258, 66)


,Unnamed: 0,rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg,rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg,dmgrphc_b_zip_code_cd,cust_prd_hld_d_vas_active_0_to_30_days_avg_val,rev_m_eom_total_bill_amt_m0_val,cust_prd_hld_d_fixedbroadband_closed_0_to_30_days_avg_val,ci_d_sum_compl_fraga_last_0_to_60_days_cnt,dmgrphc_d_tenure_in_days_val,cust_prd_hld_d_tvchannelpackage_closed_0_to_30_days_avg_val,...,cust_prd_hld_d_vas_closed_0_to_30_days_avg_val,cust_prd_hld_d_postpaid_mobilevoicesubscription_cnt,ci_d_sum_cmpl_over_contacts_in_0_30_days,ci_d_variation_sum_total_compl_last_30_over_60_days_cnt,dmgrphc_b_no_of_dependents_val,cust_prd_hld_d_tvchannelpackage_closed_30_to_60_days_avg_val,cust_prd_hld_d_tvchannelpackage_active_0_to_30_days_avg_val,cust_prd_hld_d_fixedbroadband_active_0_to_30_over_30_to_60_days_avg_val,rev_m_eom_total_bill_amt_m2_val,ci_d_total_num_of_interactions_weekend_last_0_to_30_cnt
0,6,0.0,0.0,0,0.870968,0.0,0.0,0,140,0.0,...,0.0,1,NaN,NaN,1,0.0,0.0,NaN,0.0,0
1,8,0.0,0.0,81,0.000000,0.0,0.0,0,548,0.0,...,0.0,2,NaN,NaN,3,0.0,0.0,1.000000,0.0,0
2,15,NaN,NaN,94,2.000000,NaN,0.0,0,1224,0.0,...,0.0,3,NaN,NaN,3,0.0,0.0,1.148148,NaN,0
3,16,NaN,NaN,94,2.000000,NaN,0.0,0,1224,0.0,...,0.0,3,NaN,NaN,3,0.0,0.0,1.148148,NaN,0
4,17,NaN,NaN,94,2.000000,NaN,0.0,0,1224,0.0,...,0.0,3,NaN,NaN,3,0.0,0.0,1.148148,NaN,0


In [35]:
df_X_train_fold_0 = df_X_train_fold_0.set_index(df_X_train_fold_0.columns[0])
df_X_train_fold_0

,rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg,rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg,dmgrphc_b_zip_code_cd,cust_prd_hld_d_vas_active_0_to_30_days_avg_val,rev_m_eom_total_bill_amt_m0_val,cust_prd_hld_d_fixedbroadband_closed_0_to_30_days_avg_val,ci_d_sum_compl_fraga_last_0_to_60_days_cnt,dmgrphc_d_tenure_in_days_val,cust_prd_hld_d_tvchannelpackage_closed_0_to_30_days_avg_val,ci_d_sum_total_compl_last_0_to_30_days_cnt,...,cust_prd_hld_d_vas_closed_0_to_30_days_avg_val,cust_prd_hld_d_postpaid_mobilevoicesubscription_cnt,ci_d_sum_cmpl_over_contacts_in_0_30_days,ci_d_variation_sum_total_compl_last_30_over_60_days_cnt,dmgrphc_b_no_of_dependents_val,cust_prd_hld_d_tvchannelpackage_closed_30_to_60_days_avg_val,cust_prd_hld_d_tvchannelpackage_active_0_to_30_days_avg_val,cust_prd_hld_d_fixedbroadband_active_0_to_30_over_30_to_60_days_avg_val,rev_m_eom_total_bill_amt_m2_val,ci_d_total_num_of_interactions_weekend_last_0_to_30_cnt
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
6,0.0,0.0,0,0.870968,0.000000,0.0,0,140,0.0,0,...,0.0,1,NaN,NaN,1,0.0,0.000000,NaN,0.0,0
8,0.0,0.0,81,0.000000,0.000000,0.0,0,548,0.0,0,...,0.0,2,NaN,NaN,3,0.0,0.000000,1.000000,0.0,0
15,NaN,NaN,94,2.000000,NaN,0.0,0,1224,0.0,0,...,0.0,3,NaN,NaN,3,0.0,0.000000,1.148148,NaN,0
16,NaN,NaN,94,2.000000,NaN,0.0,0,1224,0.0,0,...,0.0,3,NaN,NaN,3,0.0,0.000000,1.148148,NaN,0
17,NaN,NaN,94,2.000000,NaN,0.0,0,1224,0.0,0,...,0.0,3,NaN,NaN,3,0.0,0.000000,1.148148,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5888,0.0,0.0,10,1.000000,99.183382,0.0,0,751,0.0,0,...,0.0,4,NaN,NaN,2,0.0,0.645161,6.750000,0.0,0
5889,0.0,0.0,10,1.000000,99.183382,0.0,0,751,0.0,0,...,0.0,4,NaN,NaN,2,0.0,0.645161,6.750000,0.0,0
5890,0.0,0.0,10,1.000000,99.183382,0.0,0,751,0.0,0,...,0.0,4,NaN,NaN,2,0.0,0.645161,6.750000,0.0,0


In [36]:
df_X_train_fold_0.fillna(0)

,rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg,rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg,dmgrphc_b_zip_code_cd,cust_prd_hld_d_vas_active_0_to_30_days_avg_val,rev_m_eom_total_bill_amt_m0_val,cust_prd_hld_d_fixedbroadband_closed_0_to_30_days_avg_val,ci_d_sum_compl_fraga_last_0_to_60_days_cnt,dmgrphc_d_tenure_in_days_val,cust_prd_hld_d_tvchannelpackage_closed_0_to_30_days_avg_val,ci_d_sum_total_compl_last_0_to_30_days_cnt,...,cust_prd_hld_d_vas_closed_0_to_30_days_avg_val,cust_prd_hld_d_postpaid_mobilevoicesubscription_cnt,ci_d_sum_cmpl_over_contacts_in_0_30_days,ci_d_variation_sum_total_compl_last_30_over_60_days_cnt,dmgrphc_b_no_of_dependents_val,cust_prd_hld_d_tvchannelpackage_closed_30_to_60_days_avg_val,cust_prd_hld_d_tvchannelpackage_active_0_to_30_days_avg_val,cust_prd_hld_d_fixedbroadband_active_0_to_30_over_30_to_60_days_avg_val,rev_m_eom_total_bill_amt_m2_val,ci_d_total_num_of_interactions_weekend_last_0_to_30_cnt
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
6,0.0,0.0,0,0.870968,0.000000,0.0,0,140,0.0,0,...,0.0,1,0.0,0.0,1,0.0,0.000000,0.000000,0.0,0
8,0.0,0.0,81,0.000000,0.000000,0.0,0,548,0.0,0,...,0.0,2,0.0,0.0,3,0.0,0.000000,1.000000,0.0,0
15,0.0,0.0,94,2.000000,0.000000,0.0,0,1224,0.0,0,...,0.0,3,0.0,0.0,3,0.0,0.000000,1.148148,0.0,0
16,0.0,0.0,94,2.000000,0.000000,0.0,0,1224,0.0,0,...,0.0,3,0.0,0.0,3,0.0,0.000000,1.148148,0.0,0
17,0.0,0.0,94,2.000000,0.000000,0.0,0,1224,0.0,0,...,0.0,3,0.0,0.0,3,0.0,0.000000,1.148148,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5888,0.0,0.0,10,1.000000,99.183382,0.0,0,751,0.0,0,...,0.0,4,0.0,0.0,2,0.0,0.645161,6.750000,0.0,0
5889,0.0,0.0,10,1.000000,99.183382,0.0,0,751,0.0,0,...,0.0,4,0.0,0.0,2,0.0,0.645161,6.750000,0.0,0
5890,0.0,0.0,10,1.000000,99.183382,0.0,0,751,0.0,0,...,0.0,4,0.0,0.0,2,0.0,0.645161,6.750000,0.0,0


In [37]:
df_X_train_fold_0.values

array([[ 0.        ,  0.        ,  0.        , ...,         nan,
         0.        ,  0.        ],
       [ 0.        ,  0.        , 81.        , ...,  1.        ,
         0.        ,  0.        ],
       [        nan,         nan, 94.        , ...,  1.14814815,
                nan,  0.        ],
       ...,
       [ 0.        ,  0.        , 10.        , ...,  6.75      ,
         0.        ,  0.        ],
       [ 0.        ,  0.        , 69.        , ...,  1.67741935,
         0.        ,  0.        ],
       [        nan,         nan, 85.        , ...,  1.41935484,
                nan,  0.        ]])

In [43]:
train_y_fold_0 = '0/train_y.csv' 
train_y_fold_0_path = 's3://{}/{}'.format(bucket, train_y_fold_0)

df_train_y_fold_0 = pd.read_csv(train_y_fold_0_path, )
print(df_train_y_fold_0.shape)
df_train_y_fold_0.head()


(2258, 2)


,Unnamed: 0,tgt_xsell_cust_voice_to_fixed
0,6,0
1,8,0
2,15,0
3,16,0
4,17,0


In [44]:
df_train_y_fold_0 = df_train_y_fold_0.set_index(df_train_y_fold_0.columns[0])
print(df_train_y_fold_0.shape)
df_train_y_fold_0

(2258, 1)


,tgt_xsell_cust_voice_to_fixed
Unnamed: 0,
6,0
8,0
15,0
16,0
17,0
...,...
5888,0
5889,0
5890,1


In [45]:
df_train_y_fold_0.values

array([[0],
       [0],
       [0],
       ...,
       [1],
       [0],
       [0]])

In [46]:
df_train_y_fold_0.values.ravel()

array([0, 0, 0, ..., 1, 0, 0])

# Check inference data

## 1- preprocessing data

In [23]:
preprocessed_inf_data_path = "s3://mlops-dev-370702650160-eu-north-1-data/lifecycle/60d/customerone-inf/customerone-inf-p-0mogq7hgpkye-inference/1665060547/p1033/model-input/inference-data.csv"
preprocessed_inf_data_df = pd.read_csv(preprocessed_inf_data_path)
preprocessed_inf_data_df

,current_dt,customer_id,rev_m_bill_shock_eom_total_bill_amt_0m_to_avg_01m_flg,rev_m_bill_shock_eom_total_bill_amt_1m_to_avg_12m_flg,dmgrphc_b_zip_code_cd,cust_prd_hld_d_mobilevoicesubscription_closed_0_to_30_days_avg_val,cust_prd_hld_d_vas_active_0_to_30_days_avg_val,cust_prd_hld_d_mobilevoicesubscription_closed_30_to_60_days_avg_val,cust_prd_hld_d_fixedbroadband_closed_0_to_30_days_avg_val,ci_d_sum_total_compl_last_0_to_60_days_cnt,...,cust_prd_hld_d_vas_closed_0_to_30_over_30_to_60_days_avg_val,ci_d_sum_cmpl_resolved_over_contacts_in_0_30_days,cust_prd_hld_d_postpaid_tvchannelpackage_cnt,cust_prd_hld_d_fixedbroadband_closed_30_to_60_days_avg_val,rev_m_eom_total_bill_amt_m0_val,cust_prd_hld_d_vas_closed_0_to_30_days_avg_val,cust_prd_hld_d_fixedbroadband_closed_val,cust_prd_hld_d_fixedbroadband_closed_0_to_30_over_30_to_60_days_avg_val,cust_prd_hld_d_vas_closed_30_to_60_days_avg_val,ci_d_variation_sum_total_num_of_interactions_last_30_to_60_days_cnt
0,2021-07-21,100314,NaN,NaN,98,0.0,1.000000,0.0,0.0,0,...,1.0,0.0,1,0.0,NaN,1.000000,0,NaN,1.0,1.0
1,2021-07-21,100314,NaN,NaN,98,0.0,1.000000,0.0,0.0,0,...,1.0,0.0,1,0.0,NaN,1.000000,0,NaN,1.0,1.0
2,2021-07-15,100336,0.0,0.0,84,0.0,0.000000,0.0,0.0,0,...,NaN,0.0,1,0.0,0.000000,0.000000,0,NaN,0.0,1.0
3,2021-06-06,100433,0.0,0.0,64,0.0,1.000000,0.0,0.0,0,...,NaN,NaN,1,0.0,0.000000,0.000000,0,NaN,0.0,NaN
4,2021-06-02,100552,NaN,NaN,65,0.0,0.000000,0.0,0.0,0,...,NaN,NaN,1,0.0,NaN,0.000000,0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5891,2021-09-04,100356,0.0,0.0,27,0.0,0.967742,0.0,0.0,0,...,NaN,NaN,2,0.0,162.843261,0.032258,0,NaN,0.0,0.0
5892,2021-06-13,100608,NaN,NaN,73,0.0,2.000000,0.0,0.0,0,...,NaN,NaN,1,0.0,NaN,0.000000,0,NaN,0.0,NaN
5893,2021-07-13,100642,0.0,0.0,46,0.0,1.000000,0.0,0.0,0,...,NaN,NaN,1,0.0,272.358113,0.000000,0,NaN,0.0,NaN
5894,2021-08-11,100893,NaN,NaN,64,0.0,0.516129,0.0,0.0,0,...,NaN,NaN,0,0.0,NaN,0.483871,0,NaN,0.0,NaN


In [25]:
[print(f"col={col}, unique={preprocessed_inf_data_df[col].unique()}") for col in preprocessed_inf_data_df.columns];

col=current_dt, unique=['2021-07-21' '2021-07-15' '2021-06-06' '2021-06-02' '2021-08-09'
 '2021-07-26' '2021-07-12' '2021-06-17' '2021-05-29' '2021-08-07'
 '2021-08-26' '2021-07-08' '2021-06-30' '2021-06-18' '2021-05-30'
 '2021-08-06' '2021-08-11' '2021-08-01' '2021-08-21' '2021-06-10'
 '2021-08-04' '2021-07-04' '2021-06-12' '2021-08-15' '2021-06-25'
 '2021-06-22' '2021-08-14' '2021-06-04' '2021-08-16' '2021-09-02'
 '2021-08-20' '2021-08-17' '2021-07-05' '2021-07-27' '2021-06-28'
 '2021-05-28' '2021-06-08' '2021-06-29' '2021-07-17' '2021-06-27'
 '2021-07-03' '2021-06-07' '2021-08-19' '2021-05-27' '2021-08-29'
 '2021-06-15' '2021-06-26' '2021-07-14' '2021-07-19' '2021-07-10'
 '2021-09-03' '2021-07-11' '2021-07-22' '2021-05-25' '2021-08-02'
 '2021-08-03' '2021-08-25' '2021-07-16' '2021-08-05' '2021-06-09'
 '2021-07-23' '2021-07-31' '2021-08-31' '2021-09-06' '2021-09-09'
 '2021-09-07' '2021-08-10' '2021-08-30' '2021-08-28' '2021-05-26'
 '2021-07-30' '2021-08-13' '2021-07-09' '2021-08-08' 

# roc plot

In [44]:
import tarfile
import joblib
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, roc_auc_score, plot_roc_curve


## 1- load the trained model

In [45]:
model_path = "./model.tar.gz"

with tarfile.open(model_path) as tar:
    tar.extractall(path=".")

model_lgbm = joblib.load('./model.joblib')

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.23.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


## 2- load data

In [49]:
test_x_path = "s3://mlops-dev-370702650160-eu-north-1-data/lifecycle/60d/customerone-inf/180ebc1/2022_10_13_08_55_38/p1033/output/training/test/test_x.csv"
test_y_path = "s3://mlops-dev-370702650160-eu-north-1-data/lifecycle/60d/customerone-inf/180ebc1/2022_10_13_08_55_38/p1033/output/training/test/test_y.csv"

test_x = pd.read_csv(test_x_path)
test_y = pd.read_csv(test_y_path)

In [50]:
predictions = model_lgbm.predict(test_x.values)
predictions

array([0, 0, 0, ..., 0, 0, 0])

In [54]:
COLORS = {
    "dark purple": "#4D0572",
    "purple": "#990AE3",
    "medium purple": "#C562F8",
    "light purple": "#ECCBFD",
    "blue": "#0099FF",
    "orange": "#FF9B00",
}
plt.close("all")
fig, ax = plt.subplots(figsize=(16, 9))
ax.set_title("ROC Curve")
plot_roc_curve(model_lgbm, test_x.values, test_y, ax=ax, color=COLORS["purple"])
# plt.show()
plt.savefig('test_roc_plot.png')